# Ansible による mdx インスタンスへの TLJHrepo2docker のインストール

In [19]:
!./mdxcsv2inventory.py -g nfsserver vm1 -g ldapserver vm1 -g reverseproxy vm1 user-portal-vm-info.csv > hosts.ini

In [20]:
!cat hosts.ini

[all:vars]
ansible_user=mdxuser
ansible_remote_tmp=/tmp/.ansible
ethipv4prefix=163.220.176.0/21
rdmaipv4prefix=10.146.88.0/21
ethipv6prefix=::/64

[default]
163.220.176.59  hostname=tljh-mdx2.cs.rcos.nii.ac.jp ethipv4=163.220.176.59  rdmaipv4=10.146.91.226  

[nfsserver]
# group with regexp 'vm1'

[ldapserver]
# group with regexp 'vm1'

[reverseproxy]
# group with regexp 'vm1'

